In [ ]:
%run National_Park_Codes.ipynb

In [ ]:
import math
import urllib.parse
import urllib.request
import json
import pandas as pd

baseurl = "https://developer.nps.gov/api/v1/events?"
event_summary = []

for code, name in nationalParkDict.items():
    print(f"Fetching data for {name} ({code})...")
    para_dict = {"parkCode": code, "api_key": apikey, "dateStart": "1973-01-01"}
    url2check = baseurl + urllib.parse.urlencode(para_dict)
    
    try:
        result = urllib.request.urlopen(url2check).read()
        result_dict = json.loads(result)
        total_events = int(result_dict.get("total", 0))
        pagesize = 50
        page_number = math.ceil(total_events / pagesize)
        page = 1
        
        print(f"{name} ({code}): {total_events} events")
        
        event_count = {}  # Dictionary to store event type counts for this park by year

        while page <= page_number:
            print(f"Processing page {page} of {page_number} for {name} ({code})...")
            try:
                para_dict = {
                    "parkCode": code,
                    "api_key": apikey,
                    "pagesize": pagesize,
                    "dateStart": "1973-01-01",
                    "pagenumber": page
                }
                url2check = baseurl + urllib.parse.urlencode(para_dict)
                result = urllib.request.urlopen(url2check).read()
                result_dict = json.loads(result)

                # Process event data
                for event in result_dict.get("data", []):
                    event_type = event.get("category", "Unknown")
                    date_start = event.get("datestart", "Unknown")
                    year = date_start[:4] if date_start != "Unknown" else "Unknown"

                    # Group events by year and type
                    if year != "Unknown":
                        event_count[(year, event_type)] = event_count.get((year, event_type), 0) + 1

            except Exception as e:
                print(f"Error on page {page} for {name} ({code}): {e}")
                continue
            
            page += 1
        
        # Add the park's yearly event summary to the overall list
        for (year, event_type), count in event_count.items():
            event_summary.append({
                "Park": name,
                "ParkCode": code,
                "Year": year,
                "EventType": event_type,
                "Count": count
            })
    
    except Exception as e:
        print(f"Failed to fetch data for {name} ({code}): {e}")
        continue

# Convert summary to a DataFrame
event_summary_df = pd.DataFrame(event_summary)
event_summary_df.to_csv('event_summary.csv', index=False)
# Display or save the summarized DataFrame
print(event_summary_df)